In [1]:
!rm -rf InkubaLM-Challenge
!git clone https://github.com/melissafasol/InkubaLM-Challenge.git
%cd InkubaLM-Challenge


Cloning into 'InkubaLM-Challenge'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 371 (delta 108), reused 59 (delta 26), pack-reused 202 (from 1)
Receiving objects: 100% (371/371), 1.33 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (235/235), done.
/content/InkubaLM-Challenge


In [374]:
%cd InkubaLM-Challenge
!git pull


[Errno 2] No such file or directory: 'InkubaLM-Challenge'
/content/InkubaLM-Challenge
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 351 bytes | 117.00 KiB/s, done.
From https://github.com/melissafasol/InkubaLM-Challenge
   0c450ab..117a242  main       -> origin/main
Updating 0c450ab..117a242
Fast-forward
 utils/supplement.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install -U trl bitsandbytes

In [5]:
!pip install datasets

In [375]:
import sys
sys.path.append("..")  # Add parent directory to the path

import os
from typing import List
from pathlib import Path
import numpy as np

# DO NOT EDIT
# create submission file
import pandas as pd
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
)
from utils import (
    eval,
    model_function,
    multitask,
    experiment_logger,
    supplement
    )

import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, concatenate_datasets, Dataset, Value
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [43]:
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/InkubaLM-Challenge/Output"
os.makedirs(output_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

from huggingface_hub import login

try:
    from google.colab import userdata

    # Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
    # vars as appropriate for your system.
    # userdata.get("HF_TOKEN") indicates that the name of the token in the Colab env is HF_TOKEN
    os.environ["hf_token_2"] = userdata.get("hf_token_2")
except:
    os.environ["hf_token_2"] = "----"

login(token=os.environ["hf_token_2"])

token = os.environ["hf_token_2"]
if token == "----":
    print("⚠️ Warning: No Hugging Face token found. Some models may not load.")
else:
    login(token=token)

In [14]:
hf_token_2 = '..' # paste your token here
os.environ["HF_TOKEN"] = hf_token_2

## **Load and Explore Datasets**

In [285]:
print("# Loading datasets")
train_dataset = multitask.load_and_combine_datasets("Train")
test_dataset = multitask.load_and_combine_datasets("Test")

print("\n# Example from training dataset:")
print(train_dataset[0])

print("\n# Example from test dataset:")
print(test_dataset[0])

# Loading datasets
Common Columns: ['inputs', 'ID', 'instruction', 'langs', 'targets']
Common Columns: ['inputs', 'ID', 'instruction', 'langs', 'targets']

# Example from training dataset:
{'ID': 'ID_6aba33a1_sentiment_ dev_hausa', 'langs': 'hausa', 'instruction': 'Za ka iya tantance yanayin wannan rubutu? Bi waɗannan jagororin sharhi: Kyakkyawa: idan rubutu na nuna kyakkyawan tunani, hali, da yanayi. Korau: idan rubutu yana nuna mummunar tunani ko yanayi. Neutral: idan rubutu baya nuna kyakkyawar magana ko mara kyau kai tsaye ko a kaikaice.', 'inputs': '@user @user allah ya tsayyaba yar uwa 🎂 😍', 'targets': 'Kyakkyawa'}

# Example from test dataset:
{'ID': 'ID_f3c74c7b_sentiment_test__hausa', 'langs': 'hausa', 'instruction': "Gano ra'ayin da aka bayyana a cikin wannan rubutu. Bin waɗannan jagororin, kyakkyawa yana na rubutu na nufin kyakkyawan tunani, ɗabi'a, da motsin rai. Korau na nuna rubutu na nufin mummunan tunani ko motsin rai. Tsaka-tsaki na nuna rubutu baya nufin magana mai ky

In [286]:
print("# Converting to DataFrame and extracting task types")
train_df = train_dataset.to_pandas()
train_df['task'] = train_df.ID.apply(multitask.extract_task_from_id)

print("\n# Dataset distribution by task:")
print(train_df.task.value_counts())

# Converting to DataFrame and extracting task types

# Dataset distribution by task:
task
mt           600
sentiment    400
afrixnli     400
Name: count, dtype: int64


In [288]:
train_df.loc[train_df['task'] == 'afrixnli']

,ID,langs,instruction,inputs,targets,task
1000,ID_648d37ff_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?","Bayan nace e, ya ƙare.",0,afrixnli
1001,ID_f96a39cb_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ba zan damu ba idan ɗaukan nauyin kuɗin da ake...,0,afrixnli
1002,ID_4c5d953b_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?","Alas, hakuweza kuelewa wazi kutokana na kizuiz...",1,afrixnli
1003,ID_99a61e3d_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Nace a'a amma yaci gaba da ja-in-ja.,2,afrixnli
1004,ID_a38aa9d8_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Kanuni haziwezi kuamua jinsi ya kutengeneza mk...,1,afrixnli
...,...,...,...,...,...,...
1395,ID_085354e1_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Maduka ya habari ya kitaifa hufanya maeneo yet...,1,afrixnli
1396,ID_586e104a_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Kurasa zilihusisha wanachama na maafisa wa kaw...,0,afrixnli
1397,ID_b871ea53_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ban damu ba da abinda labarun ƙasa ke nuni cik...,2,afrixnli
1398,ID_70aae970_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ya zabi ƙin kama hannayen sa saboda anyi musu ...,0,afrixnli


Analyse target sequence lengths (without balancing)

In [20]:
unbalanced_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))

print("\n# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented")
print("in the loss function compared to tasks with longer outputs (like mt).")


# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented
in the loss function compared to tasks with longer outputs (like mt).


In [90]:
print("\n# Applying target length balancing fix")
balanced_df = multitask.balance_target_lengths(train_df)

print("\n# Analyzing target sequence lengths after balancing")
task_stats_after = multitask.analyze_task_lengths(balanced_df)



# Applying target length balancing fix

# Analyzing target sequence lengths after balancing
Target sequence length analysis by task:
Task: sentiment
count    400.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: mt
count    600.000000
mean      11.801667
std        8.347163
min        2.000000
25%        6.000000
50%       10.000000
75%       15.000000
max       62.000000
Name: targets, dtype: float64
--------------------------------------------------
Task: afrixnli
count    400.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------


In [376]:
sentiment_swa_supp = supplement.generate_swahili_supplement()
sentiment_hau_supp = supplement.generate_hausa_supplement()

In [377]:
sent_df_supp = pd.concat([sentiment_swa_supp,sentiment_hau_supp])

In [378]:
sent_df_supp

,ID,task,langs,instruction,inputs,targets
0,ID_835e9cfb_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Tunakupongeza kwa jitihada zako.,Chanya
1,ID_8c705ffe_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Ujumbe huu unatia matumaini.,Chanya
2,ID_3611c473_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Napenda mtazamo wako.,Chanya
3,ID_79e5c1cf_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Mkutano huo ulikuwa wa mafanikio.,Chanya
4,ID_9363e4d8_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Unafanya vizuri sana.,Chanya
...,...,...,...,...,...,...
295,ID_be084ece_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina zaune a gida ina kallon talabijin.,Tsaka-tsaki
296,ID_14233f49_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina jiran zuwan motar haya.,Tsaka-tsaki
297,ID_b9098b1e_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina zaune a gida ina kallon talabijin.,Tsaka-tsaki
298,ID_f44b2ec6_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Yau rana ce mai kyau a gari.,Tsaka-tsaki


In [338]:
sent_df_supp.head()

,ID,task,langs,instruction,inputs,targets
0,ID_ee42a223_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Siku yangu imekuwa bora kwa sababu yako.,Chanya
1,ID_b3898e93_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Siku yangu imekuwa bora kwa sababu yako.,Chanya
2,ID_b1927467_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Tunashukuru sana kwa jitihada zako.,Chanya
3,ID_35305416_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Tunakupongeza kwa jitihada zako.,Chanya
4,ID_b053e509_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Hii ni habari njema sana.,Chanya


In [339]:
train_df.head()

,ID,task,langs,data_source,instruction,inputs,targets,premise
0,ID_6aba33a1_sentiment_ dev_hausa,sentiment,hausa,afrisenti,Za ka iya tantance yanayin wannan rubutu? Bi w...,@user @user allah ya tsayyaba yar uwa 🎂 😍,Kyakkyawa,
1,ID_ce64d307_sentiment_ dev_hausa,sentiment,hausa,naijasenti,Da fatan za a gano ra'ayin da ke cikin wannan ...,@user intenet a masallachi😭😭😭 wani salo ne na ...,Tsaka-tsaki,
2,ID_dfb02831_sentiment_ dev_swahili,sentiment,swahili,swahili_tweet,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,picha mbunge wa kilombero peter lijualikali ak...,Wastani,
3,ID_2efc9515_sentiment_ dev_hausa,sentiment,hausa,afrisenti,Gano ra'ayin da aka bayyana a cikin wannan rub...,@user @user @user @user @user hhh amma rahama ...,Tsaka-tsaki,
4,ID_ad1d9888_sentiment_ dev_swahili,sentiment,swahili,afrisenti,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,swali zuri sana nawatafuta wajuzi wa mambo wat...,Wastani,


In [379]:
required_columns = ['ID', 'task', 'langs', 'instruction', 'inputs', 'targets', 'data_source', 'premise']

for df in [train_df, sent_df_supp]:
    for col in required_columns:
        if col not in df.columns:
            df[col] = ""  # fill missing columns with empty string

# Reorder columns to be consistent
df1 = train_df[required_columns]
df2 = sent_df_supp[required_columns]

# Concatenate without NaNs
combined_df = pd.concat([df1, df2], ignore_index=True)

In [380]:
combined_df

,ID,task,langs,instruction,inputs,targets,data_source,premise
0,ID_6aba33a1_sentiment_ dev_hausa,sentiment,hausa,Za ka iya tantance yanayin wannan rubutu? Bi w...,@user @user allah ya tsayyaba yar uwa 🎂 😍,Kyakkyawa,afrisenti,
1,ID_ce64d307_sentiment_ dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,@user intenet a masallachi😭😭😭 wani salo ne na ...,Tsaka-tsaki,naijasenti,
2,ID_dfb02831_sentiment_ dev_swahili,sentiment,swahili,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,picha mbunge wa kilombero peter lijualikali ak...,Wastani,swahili_tweet,
3,ID_2efc9515_sentiment_ dev_hausa,sentiment,hausa,Gano ra'ayin da aka bayyana a cikin wannan rub...,@user @user @user @user @user hhh amma rahama ...,Tsaka-tsaki,afrisenti,
4,ID_ad1d9888_sentiment_ dev_swahili,sentiment,swahili,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,swali zuri sana nawatafuta wajuzi wa mambo wat...,Wastani,afrisenti,
...,...,...,...,...,...,...,...,...
1895,ID_be084ece_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina zaune a gida ina kallon talabijin.,Tsaka-tsaki,,
1896,ID_14233f49_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina jiran zuwan motar haya.,Tsaka-tsaki,,
1897,ID_b9098b1e_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina zaune a gida ina kallon talabijin.,Tsaka-tsaki,,
1898,ID_f44b2ec6_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Yau rana ce mai kyau a gari.,Tsaka-tsaki,,


In [386]:
def load_dataset_by_tag(dataset_type, tag, split='train'):
    return load_dataset(f"{dataset_type}{tag}", split=split)

def load_and_combine_datasets(tag, split='train'):
    """
    Load and combine multiple datasets with all unique columns (union).
    Empty strings are used for missing values.

    Args:
        tag (str): Tag for the datasets (Train, Test)
        split (str): Split to load (train, test)

    Returns:
        Dataset: Combined dataset with all unique columns
    """
    # Load datasets with the given split
    se_dataset = load_dataset_by_tag("lelapa/Sentiment", tag, split)
    mt_dataset = load_dataset_by_tag("lelapa/MT", tag, split)
    xn_dataset = load_dataset_by_tag("lelapa/XNLI", tag, split)

    # Identify all unique columns (union)
    all_columns = list(set(se_dataset.column_names) |
                      set(mt_dataset.column_names) |
                      set(xn_dataset.column_names))
    print(f"All Columns: {all_columns}")

    # Function to ensure dataset has all columns, filling missing ones with empty strings
    def ensure_all_columns(dataset, all_cols):
        # Add each missing column one by one
        for col in all_cols:
            if col not in dataset.column_names:
                # Create array of empty strings with the same length as the dataset
                empty_column = [""] * len(dataset)
                dataset = dataset.add_column(col, empty_column)

        return dataset

    # Ensure all datasets have all columns
    se_dataset = ensure_all_columns(se_dataset, all_columns)
    mt_dataset = ensure_all_columns(mt_dataset, all_columns)
    xn_dataset = ensure_all_columns(xn_dataset, all_columns)

    # Make sure 'targets' column is string type if it exists in all datasets
    if "targets" in all_columns:
        se_dataset = se_dataset.cast_column("targets", Value("string"))
        mt_dataset = mt_dataset.cast_column("targets", Value("string"))
        xn_dataset = xn_dataset.cast_column("targets", Value("string"))

    # Concatenate datasets
    combined_dataset = concatenate_datasets([se_dataset, mt_dataset, xn_dataset])

    return combined_dataset


def extract_task_from_id(id_string):
    """
    Extract task type from ID string.

    Args:
        id_string (str): ID string containing task information

    Returns:
        str: Extracted task type
    """
    task = id_string.split('_')[3]
    # Handle special case for sentiment task
    return 'sentiment' if task == ' dev' else task

def extract_task_from_id_test(id_string):
    """
    Extract task type from ID string.

    Args:
        id_string (str): ID string containing task information

    Returns:
        str: Extracted task type
    """
    parts = id_string.split('_')
    task = id_string.split('_')[3]
    if len(parts) >= 4:
        return parts[2]  # 'sentiment', 'afrixnli', etc.
    else:
        return task


def analyze_task_lengths(df, task_column='task'):
    """
    Analyze and display target sequence lengths for each task.

    Args:
        df (DataFrame): DataFrame containing task and targets columns
        task_column (str): Name of the task column

    Returns:
        dict: Statistics about target lengths per task
    """
    stats = {}
    print("Target sequence length analysis by task:")
    print("="*50)

    for task in df[task_column].unique():
        mask = df[task_column] == task
        task_stats = df.loc[mask, 'targets'].apply(lambda x: len(x.split())).describe()
        print(f"Task: {task}")
        print(task_stats)
        print("-"*50)
        stats[task] = task_stats

    return stats


def balance_target_lengths(df, task_column='task', reference_task='mt', repetition_factor=11):
    """
    Balance target sequence lengths by repeating shorter targets.

    Args:
        df (DataFrame): DataFrame containing task and targets columns
        task_column (str): Name of the task column
        reference_task (str): Task with longer sequences to use as reference
        repetition_factor (int): Number of times to repeat shorter sequences

    Returns:
        DataFrame: DataFrame with balanced target lengths
    """
    df_balanced = df.copy()

    for task in df_balanced[task_column].unique():
        if task != reference_task:
            mask = df_balanced[task_column] == task
            df_balanced.loc[mask, 'targets'] = df_balanced.loc[mask, 'targets'].apply(
                lambda x: ' '.join([x] * repetition_factor)
            )

    return df_balanced


def plot_target_lengths(df_before, df_after, task_column='task'):
    """
    Plot target lengths before and after balancing.

    Args:
        df_before (DataFrame): DataFrame before balancing
        df_after (DataFrame): DataFrame after balancing
        task_column (str): Name of the task column
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Before balancing
    task_stats_before = {}
    for task in df_before[task_column].unique():
        mask = df_before[task_column] == task
        lengths = df_before.loc[mask, 'targets'].apply(lambda x: len(x.split()))
        task_stats_before[task] = lengths.mean()

    # After balancing
    task_stats_after = {}
    for task in df_after[task_column].unique():
        mask = df_after[task_column] == task
        lengths = df_after.loc[mask, 'targets'].apply(lambda x: len(x.split()))
        task_stats_after[task] = lengths.mean()

    # Plotting
    ax1.bar(task_stats_before.keys(), task_stats_before.values())
    ax1.set_title('Average Target Length Before Balancing')
    ax1.set_ylabel('Average number of tokens')

    ax2.bar(task_stats_after.keys(), task_stats_after.values())
    ax2.set_title('Average Target Length After Balancing')

    plt.tight_layout()
    #plt.savefig('target_length_comparison.png')
    plt.show()
    #plt.close()


def formatting_prompts_func(example):
    """
    Format examples for instruction tuning.

    Args:
        example (dict): Example containing instruction, inputs, and targets

    Returns:
        str: Formatted prompt
    """
    premise = example['premise']
    premise = premise+'\n' if len(premise) else ''
    if example['targets'] is not None:
        return f"### Instruction: {example['instruction']}\n### Input: {premise}{example['inputs']}\n### Response: {example['targets']}"
    return f"### Instruction: {example['instruction']}\n### Input: {premise}{example['inputs']}\n### Response:"


def setup_model_and_tokenizer(model_name, use_4bit=True):
    """
    Set up model and tokenizer for QLoRA fine-tuning.

    Args:
        model_name (str): Name of the base model
        use_4bit (bool): Whether to use 4-bit quantization

    Returns:
        tuple: (model, tokenizer, bnb_config)
    """
    # Define BitsAndBytes config for quantization
    if use_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
    else:
        bnb_config = None

    # Load model with quantization config
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        token=token,
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer, bnb_config


def apply_lora_adapters(model, r=8, lora_alpha=16, dropout=0.05):
    """
    Apply LoRA adapters to the model.

    Args:
        model: Base model
        r (int): LoRA rank
        lora_alpha (int): LoRA alpha parameter
        dropout (float): Dropout probability

    Returns:
        model: Model with LoRA adapters
    """
    # Define LoRA Config
    lora_config = LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Apply LoRA adapters to model
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    return model


def setup_trainer(model, dataset, tokenizer, output_dir, num_epochs=3):
    """
    Set up SFTTrainer for direct fine-tuning.

    Args:
        model: Model to fine-tune
        dataset: Training dataset
        tokenizer: Tokenizer
        output_dir (str): Output directory for checkpoints
        num_epochs (int): Number of training epochs

    Returns:
        SFTTrainer: Trainer object
    """
    # Define response template for proper label masking
    response_template_with_context = "\n### Response:"
    response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[2:]

    # Data collator for masked LM training
    collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

    # Training arguments
    train_args = SFTConfig(
        output_dir=output_dir,
        max_seq_length=256,
        num_train_epochs=num_epochs,
        save_strategy="epoch",
        optim = 'adamw_bnb_8bit',
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        logging_steps=10,
        save_total_limit=2,
        report_to=[],  # Disable wandb
    )

    # Trainer setup
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        args=train_args,
        formatting_func=formatting_prompts_func,
        data_collator=collator,
    )

    return trainer


def generate_response(model, tokenizer, prompt, max_new_tokens=20):
    """
    Generate response using fine-tuned model.

    Args:
        model: Fine-tuned model
        tokenizer: Tokenizer
        prompt (str): Input prompt
        max_new_tokens (int): Maximum number of tokens to generate

    Returns:
        str: Generated response
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text[len(prompt):].strip()

    return response


def encode_sentiment_label(label):
    """
    Encode sentiment label to integer.

    Args:
        label (str): Sentiment label

    Returns:
        int: Encoded label
    """
    for c, i in enumerate(["Chanya", "Wastani", "Hasi"]):
        if label == i:
            return c
    for c, i in enumerate(["Kyakkyawa", "Tsaka-tsaki", "Korau"]):
        if label == i:
            return c
    return 0


def apply_inference_to_test_data(model, tokenizer, test_dataset):
    """
    Apply inference to test dataset.

    Args:
        model: Fine-tuned model
        tokenizer: Tokenizer
        test_dataset: Test dataset

    Returns:
        DataFrame: DataFrame with generated responses
    """
    df = pd.DataFrame(test_dataset)
    model.eval()

    # Apply inference with tqdm progress bar
    tqdm.pandas(desc="Generating Responses")
    df['generated'] = df.progress_apply(
        lambda row: generate_response(model, tokenizer, formatting_prompts_func(row)),
        axis=1
    )

    # Process responses based on task type
    df['Response'] = ''

    # Sentiment task
    mask = df.ID.apply(lambda x: 'sentiment' in x)
    df.loc[mask, 'Response'] = df.loc[mask, 'generated'].apply(
        lambda x: encode_sentiment_label(x.strip().split()[0])
    )

    # XNLI task
    mask = df.ID.apply(lambda x: 'afrixnli' in x)
    df.loc[mask, 'Response'] = df.loc[mask, 'generated'].apply(
        lambda x: int(x.strip().split()[0])%3 if x.strip().split()[0].isdigit() else 0
    )

    # MT task
    mask = df.ID.apply(lambda x: 'mt_' in x)
    df.loc[mask, 'Response'] = df.loc[mask, 'generated']

    return df

def display_formatted_examples(df, num_examples=2):
    """
    Display formatted examples for each task.

    Args:
        df (DataFrame): DataFrame containing the examples
        num_examples (int): Number of examples to display per task
    """
    for task in df.task.unique():
        print(f"\n\n{'='*40}\nTask: {task}\n{'='*40}")
        mask = df.task == task
        for i, (_, row) in enumerate(df[mask].iterrows()):
            if i >= num_examples:
                break

            print(f"\nExample {i+1}:")
            print("-" * 40)
            formatted = formatting_prompts_func(row)
            print(formatted)
            print("-" * 40)

In [325]:
print("# Loading datasets")
train_dataset = load_and_combine_datasets("Train")
test_dataset = load_and_combine_datasets("Test")

print("\n# Example from training dataset:")
print(train_dataset[0])

print("\n# Example from test dataset:")
print(test_dataset[0])

# Loading datasets
All Columns: ['premise', 'inputs', 'data_source', 'ID', 'instruction', 'task', 'langs', 'targets']
All Columns: ['premise', 'inputs', 'data_source', 'ID', 'instruction', 'task', 'langs', 'targets']

# Example from training dataset:
{'ID': 'ID_6aba33a1_sentiment_ dev_hausa', 'task': 'sentiment', 'langs': 'hausa', 'data_source': 'afrisenti', 'instruction': 'Za ka iya tantance yanayin wannan rubutu? Bi waɗannan jagororin sharhi: Kyakkyawa: idan rubutu na nuna kyakkyawan tunani, hali, da yanayi. Korau: idan rubutu yana nuna mummunar tunani ko yanayi. Neutral: idan rubutu baya nuna kyakkyawar magana ko mara kyau kai tsaye ko a kaikaice.', 'inputs': '@user @user allah ya tsayyaba yar uwa 🎂 😍', 'targets': 'Kyakkyawa', 'premise': ''}

# Example from test dataset:
{'ID': 'ID_f3c74c7b_sentiment_test__hausa', 'task': 'sentiment', 'langs': 'hausa', 'data_source': 'afrisenti', 'instruction': "Gano ra'ayin da aka bayyana a cikin wannan rubutu. Bin waɗannan jagororin, kyakkyawa yan

In [388]:
train_df = train_dataset.to_pandas()
train_df_sent_supp = pd.concat([sent_df_supp,train_df])

In [391]:
print("# Converting to DataFrame and extracting task types")
#train_df = train_dataset.to_pandas()
#train_df['task'] = train_df.ID.apply(extract_task_from_id)
train_df['task'] = train_df.ID.apply(extract_task_from_id)
train_df_sent_supp = pd.concat([train_df, sent_df_supp])

print("\n# Dataset distribution by task:")
print(train_df_sent_supp.task.value_counts())

# Converting to DataFrame and extracting task types

# Dataset distribution by task:
task
sentiment    900
mt           600
afrixnli     400
Name: count, dtype: int64


In [392]:
print("\n# Applying target length balancing fix")
#balanced_df = balance_target_lengths(train_df)
balanced_df = balance_target_lengths(train_df_sent_supp)


# Applying target length balancing fix


In [393]:
balanced_dataset = Dataset.from_pandas(balanced_df.reset_index(drop=True))

In [394]:
model_name = "lelapa/InkubaLM-0.4B"
# turn off qlora: use_4bit=False
model, tokenizer, bnb_config = setup_model_and_tokenizer(model_name, use_4bit=False)

In [395]:
balanced_dataset

Dataset({
    features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'premise'],
    num_rows: 1900
})

In [396]:
balanced_trainer = setup_trainer(
    model=model,
    dataset=balanced_dataset,
    tokenizer=tokenizer,
    output_dir="./sft_model/balanced_supp"
)
balanced_trainer.train()

Applying formatting function to train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1900 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [262]:
output_dir = '/content/drive/MyDrive/InkubaLM/outputs/'

In [265]:
# Save base model (optional, if not already public)
#model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/base_model"))

# Save LoRA adapter
model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter"))
tokenizer.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter"))


('/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter/tokenizer.model',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter/added_tokens.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter/tokenizer.json')

In [267]:
ckpt_path = os.path.join(output_dir, "sft_model/balanced/checkpoint-261")
os.makedirs(ckpt_path, exist_ok=True)
#inference_model = AutoModelForCausalLM.from_pretrained(ckpt_path,device_map="auto")
inference_model = model.eval()
results_df = apply_inference_to_test_data(inference_model, tokenizer, test_dataset)


Generating Responses:   0%|          | 0/900 [00:00<?, ?it/s]

,ID,generated,Response
0,ID_f3c74c7b_sentiment_test__hausa,Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki,1
1,ID_aad19dbf_sentiment_test__hausa,Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa,0
2,ID_f6de0381_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2
3,ID_cbec84fe_sentiment_test__swahili,Wastani Wastani Wastani Wastani Wastani Wastan...,1
4,ID_885caf5c_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2


In [268]:
results_df[['ID', 'Response']].to_csv(os.path.join(ckpt_path,'submission_full_finetune.csv'), index=False)
results_df[['ID', 'generated','Response']].head()

,ID,generated,Response
0,ID_f3c74c7b_sentiment_test__hausa,Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki,1
1,ID_aad19dbf_sentiment_test__hausa,Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa,0
2,ID_f6de0381_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2
3,ID_cbec84fe_sentiment_test__swahili,Wastani Wastani Wastani Wastani Wastani Wastan...,1
4,ID_885caf5c_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2


In [273]:
submission_csv = results_df[['ID','Response']]

In [277]:
submission_csv.to_csv(os.path.join(ckpt_path,'submission_full_finetune.csv'), index=False)

In [278]:
test_csv = pd.read_csv(os.path.join(ckpt_path,'submission_full_finetune.csv'))

In [279]:
test_csv

,ID,Response
0,ID_f3c74c7b_sentiment_test__hausa,1
1,ID_aad19dbf_sentiment_test__hausa,0
2,ID_f6de0381_sentiment_test__hausa,2
3,ID_cbec84fe_sentiment_test__swahili,1
4,ID_885caf5c_sentiment_test__hausa,2
...,...,...
895,ID_88cd08fe_test__afrixnli_hau,0
896,ID_404bc9af_test__afrixnli_swa,0
897,ID_a399ed32_test__afrixnli_swa,0
898,ID_9f28a032_test__afrixnli_hau,0
